In [1]:
from bitcoinlib.keys import Key, sign, verify
from collections import defaultdict
import hashlib
import os
import re

# Blockchain structure
# [[previous_hashblock, hashblock, transactions, admin_sign], next_block]
# transactions = [{amount, type_transaction, user, user_sign, user_destinatary (optional)}, next_transaction]
# type_transactions = create, transfer

def check_blocks(blocks, admin_pk):
    users_history = defaultdict(float)

    if blocks[0][0] != "000000000000000":
        print("Error: The initial block does not have a valid hash.")
        return False

    for i, current_block in enumerate(blocks):
        if i > 0 and current_block[0] != blocks[i - 1][1]:
            print(f"Error: Invalid link between block {i - 1} and block {i}.")
            return False
        
        block_data = current_block[0] + "".join(
            [
                f"{tx['amount']}{tx['type_transaction']}{tx['user']}"
                + (str(tx.get("user_destinatary", "")) if tx['type_transaction'] == "transfer" else "")
                for tx in current_block[2]
            ]
        )
        block_hash = hashlib.sha256(block_data.encode('utf-8')).hexdigest()
        if not verify(block_hash, current_block[3], admin_pk):
            print(f"Error: Invalid administrator signature on block {i}.")
            return False

        for transaction in current_block[2]:
            if "amount" not in transaction or "type_transaction" not in transaction or "user" not in transaction:
                print(f"Error: Missing fields in a transaction in block {i}.")
                return False

            tx_data = f"{transaction['amount']}{transaction['type_transaction']}{transaction['user']}"
            if transaction["type_transaction"] == "transfer":
                if "user_destinatary" not in transaction or transaction["user_destinatary"] is None:
                    print(f"Error: Missing 'user_destinatary' in transfer transaction in block {i}.")
                    return False
                tx_data += str(transaction["user_destinatary"])

                if "user_sign" not in transaction or transaction["user_sign"] is None:
                    print(f"Error: Missing 'user_sign' in transfer transaction in block {i}.")
                    return False

                computed_hash = hashlib.sha256((current_block[0] + tx_data).encode('utf-8')).hexdigest()
                if not verify(computed_hash, transaction["user_sign"], transaction["user"]):
                    print(f"Error: Invalid user signature in a transaction in block {i}.")
                    return False

            if transaction["type_transaction"] == "create":
                users_history[transaction["user"]] += transaction["amount"]
            elif transaction["type_transaction"] == "transfer":
                users_history[transaction["user"]] -= transaction["amount"]
                users_history[transaction["user_destinatary"]] += transaction["amount"]

                if users_history[transaction["user"]] < 0:
                    print(f"Error: Insufficient balance for user {transaction['user']} in block {i}.")
                    return False

    return users_history


def create_block(transactions, admin_sk, admin_pk, blocks=[]):
    if not transactions:
        print("Error: No transactions provided to create a block.")
        return False
    
    blocks = blocks or []
    block = [None, None, [], None]
    block[0] = "000000000000000" if not blocks else blocks[-1][1]

    for tx in transactions:
        if "amount" not in tx or "type_transaction" not in tx or "user" not in tx:
            print("Error: Missing transaction fields.")
            return False

        tx_data = f"{tx['amount']}{tx['type_transaction']}{tx['user']}"
        if tx["type_transaction"] == "transfer" and "user_destinatary" in tx:
            tx_data += str(tx["user_destinatary"])
            tx_hash = hashlib.sha256((block[0] + tx_data).encode('utf-8')).hexdigest()
            tx["user_sign"] = sign(tx_hash, tx["user_sk"])
        elif tx["type_transaction"] == "transfer":
            print("Error: Transfer transaction missing 'user_destinatary'.")
            return False
        
        block[2].append({
            "amount": tx["amount"],
            "type_transaction": tx["type_transaction"],
            "user": tx["user"],
            "user_sign": tx.get("user_sign", None),
            "user_destinatary": tx.get("user_destinatary", None)
        })

    block_data = block[0] + "".join(
        [
            f"{tx['amount']}{tx['type_transaction']}{tx['user']}"
            + (str(tx.get("user_destinatary", "")) if tx['type_transaction'] == "transfer" else "")
            for tx in block[2]
        ]
    )
    block[1] = hashlib.sha256(block_data.encode('utf-8')).hexdigest()
    block[3] = sign(block[1], admin_sk)

    blocks.append(block)
    if check_blocks(blocks, admin_pk):
        return blocks
    else:
        print("Error: Block validation failed after adding the new block.")
        return False


def save_block_to_file(block, block_index, filename, admin_sign):
    with open(filename, "w") as f:
        f.write(f"Block {block_index}:\n")
        f.write(f"Previous hash = {block[0]}\n")
        f.write(f"Current hash = {block[1]}\n")
        for tx in block[2]:
            if tx["type_transaction"] == "create":
                f.write(f"{tx['amount']} coins are created for {tx['user']}\n")
            elif tx["type_transaction"] == "transfer":
                f.write(f"{tx['user']} sends {tx['amount']} coins to {tx['user_destinatary']} (Signature: {tx['user_sign']})\n")
        f.write(f"(Admin Signature: {admin_sign})\n")



def publish_blocks(blocks):
    os.makedirs("blocks", exist_ok=True)
    for i, block in enumerate(blocks):
        filename = f"blocks/block_{i}.txt"
        save_block_to_file(block, i, filename, str(block[3]))
        print(f"Block {i} saved in: {filename}")


def print_blocks(blocks, details=True):
    for i, block in enumerate(blocks):
        print(f"Block {i} --------------------------------------------------------------------------------------------")
        print("Previous hash:", block[0])
        print("Current hash:", block[1])
        if details:
            for j, tx in enumerate(block[2]):
                print(f"  Transaction {j + 1}:")
                for key, value in tx.items():
                    print(f"    {key}: {value}")

def parse_transaction(line):
    create_pattern = r"(\d+) coins are created for ([a-f0-9]+)"
    transfer_pattern = r"([a-f0-9]+) sends (\d+) coins to ([a-f0-9]+) \(Signature: (.*?)\)"
    
    create_match = re.search(create_pattern, line)
    transfer_match = re.search(transfer_pattern, line)
    
    if create_match:
        amount = int(create_match.group(1))
        user = create_match.group(2)
        return {
            "amount": amount,
            "type_transaction": "create",
            "user": Key(user).public(),
            "user_sign": None,
            "user_destinatary": None
        }
    
    if transfer_match:
        user = transfer_match.group(1)
        amount = int(transfer_match.group(2))
        user_destinatary = transfer_match.group(3)
        user_sign = transfer_match.group(4)
        return {
            "amount": amount,
            "type_transaction": "transfer",
            "user": Key(user).public(),
            "user_sign": user_sign,
            "user_destinatary": Key(user_destinatary).public()
        }

    return None


def parse_block(block_text):
    block = []

    previous_hash_pattern = r"Previous hash = ([a-f0-9]+)"
    current_hash_pattern = r"Current hash = ([a-f0-9]+)"
    
    previous_hash = re.search(previous_hash_pattern, block_text)
    current_hash = re.search(current_hash_pattern, block_text)
    
    if previous_hash and current_hash:
        previous_hashblock = previous_hash.group(1)
        hashblock = current_hash.group(1)
    else:
        return None

    transactions = []
    lines = block_text.splitlines()
    for line in lines:
        transaction = parse_transaction(line)
        if transaction:
            transactions.append(transaction)

    admin_sign_pattern = r"Admin Signature: (.*?)(?=\))"
    admin_sign = re.search(admin_sign_pattern, block_text)
    if admin_sign:
        admin_sign_value = admin_sign.group(1)
    
    block.append(previous_hashblock)
    block.append(hashblock)
    block.append(transactions)
    block.append(admin_sign_value)
    
    return block


def read_blocks_from_directory(directory_path):
    blocks = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            with open(os.path.join(directory_path, filename), "r") as file:
                block_text = file.read()
                block = parse_block(block_text)
                if block:
                    blocks.append(block)
    return blocks


In [2]:
from bitcoinlib.keys import Key
admin_k = Key()
admin_sk, admin_pk = admin_k.secret, admin_k.public()
user1_k = Key()
user1_sk, user1_pk = user1_k.secret, user1_k.public()
user2_k = Key()
user2_sk, user2_pk = user2_k.secret, user2_k.public()
user3_k = Key()
user3_sk, user3_pk = user3_k.secret, user3_k.public()

blocks = []

transactions_block_0 = [
    {"amount": 10, "type_transaction": "create", "user": user1_pk},
]

transactions_block_1 = [
    {"amount": 3, "type_transaction": "transfer", "user": user1_pk, "user_sk": user1_sk, "user_destinatary": user2_pk},
]

transactions_block_2 = [
    {"amount": 4, "type_transaction": "transfer", "user": user1_pk, "user_sk": user1_sk, "user_destinatary": user3_pk},
    {"amount": 2, "type_transaction": "transfer", "user": user2_pk, "user_sk": user2_sk, "user_destinatary": user3_pk},
]

blocks = create_block(transactions_block_0, admin_sk, admin_pk, blocks=blocks)
blocks = create_block(transactions_block_1, admin_sk, admin_pk, blocks=blocks)
blocks = create_block(transactions_block_2, admin_sk, admin_pk, blocks=blocks)

publish_blocks(blocks)

print_blocks(blocks)

Block 0 saved in: blocks/block_0.txt
Block 1 saved in: blocks/block_1.txt
Block 2 saved in: blocks/block_2.txt
Block 0 --------------------------------------------------------------------------------------------
Previous hash: 000000000000000
Current hash: 01816c0d7bdea9c38259089b9c9e4fd9a409832d6f6334dbc94d16566834c864
  Transaction 1:
    amount: 10
    type_transaction: create
    user: 035025a9c7557afdf5e7dfdab43a52b4d2d9ce4498a135638b51adffe21a4a2d77
    user_sign: None
    user_destinatary: None
Block 1 --------------------------------------------------------------------------------------------
Previous hash: 01816c0d7bdea9c38259089b9c9e4fd9a409832d6f6334dbc94d16566834c864
Current hash: 878e5b22b647588f514185c88c10f2bca5b62fcfa34caad502e51fde456745ce
  Transaction 1:
    amount: 3
    type_transaction: transfer
    user: 035025a9c7557afdf5e7dfdab43a52b4d2d9ce4498a135638b51adffe21a4a2d77
    user_sign: 3045022100893d205f9a20f9f888defb34572843cbf292ca08874c40d77d229c58096c0daf02204

In [3]:
loaded_blocks = read_blocks_from_directory("blocks/")

print_blocks(loaded_blocks)
if check_blocks(loaded_blocks, admin_pk):
    print("All blocks are valid.")
else:
    print("The blocks contain errors.")

Block 0 --------------------------------------------------------------------------------------------
Previous hash: 000000000000000
Current hash: 01816c0d7bdea9c38259089b9c9e4fd9a409832d6f6334dbc94d16566834c864
  Transaction 1:
    amount: 10
    type_transaction: create
    user: 035025a9c7557afdf5e7dfdab43a52b4d2d9ce4498a135638b51adffe21a4a2d77
    user_sign: None
    user_destinatary: None
Block 1 --------------------------------------------------------------------------------------------
Previous hash: 01816c0d7bdea9c38259089b9c9e4fd9a409832d6f6334dbc94d16566834c864
Current hash: 878e5b22b647588f514185c88c10f2bca5b62fcfa34caad502e51fde456745ce
  Transaction 1:
    amount: 3
    type_transaction: transfer
    user: 035025a9c7557afdf5e7dfdab43a52b4d2d9ce4498a135638b51adffe21a4a2d77
    user_sign: 3045022100893d205f9a20f9f888defb34572843cbf292ca08874c40d77d229c58096c0daf02204187b7588d2217d033c2447f84a250ebe70d1931c0bc1276481b040c3780dd9c01
    user_destinatary: 02a2c465e7ec970f15a96b4